# Library

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import BallTree

# Functions

In [4]:
def generateSingleLabel(multi_label):
    

    if ("Beer, Wine & Spirits, " in multi_label):
        single_label = multi_label.replace("Beer, Wine & Spirits, ", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
    elif (", Beer, Wine & Spirits, " in multi_label):
        single_label = multi_label.replace(", Beer, Wine & Spirits, ", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
    elif (", Beer, Wine & Spirits" in multi_label):
        single_label = multi_label.replace(", Beer, Wine & Spirits", '')
        single_label = single_label.split(', ')
        single_label.append("Beer, Wine & Spirits")
                                           
    elif ("Wills, Trusts, & Probates, " in multi_label):
        single_label = multi_label.replace("Wills, Trusts, & Probates, ", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
    elif (", Wills, Trusts, & Probates, " in multi_label):
        single_label = multi_label.replace(", Wills, Trusts, & Probates, ", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
    elif (", Wills, Trusts, & Probates" in multi_label):
        single_label = multi_label.replace(", Wills, Trusts, & Probates", '')
        single_label = single_label.split(', ')
        single_label.append("Wills, Trusts, & Probates")
                                           
    elif ("Books, Mags, Music & Video, " in multi_label):
        single_label = multi_label.replace("Books, Mags, Music & Video, ", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
    elif (", Books, Mags, Music & Video, " in multi_label):
        single_label = multi_label.replace(", Books, Mags, Music & Video, ", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
    elif (", Books, Mags, Music & Video" in multi_label):
        single_label = multi_label.replace(", Books, Mags, Music & Video", '')
        single_label = single_label.split(', ')
        single_label.append("Books, Mags, Music & Video")
                                           
    elif ("Used, Vintage & Consignment, " in multi_label):
        single_label = multi_label.replace("Used, Vintage & Consignment, ", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
    elif (", Used, Vintage & Consignment, " in multi_label):
        single_label = multi_label.replace(", Used, Vintage & Consignment, ", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
    elif (", Used, Vintage & Consignment" in multi_label):
        single_label = multi_label.replace(", Used, Vintage & Consignment", '')
        single_label = single_label.split(', ')
        single_label.append("Used, Vintage & Consignment")
        
    
    else:
        
        if (multi_label == "Beer, Wine & Spirits" or
           multi_label == "Wills, Trusts, & Probates" or
           multi_label == "Books, Mags, Music & Video" or
           multi_label == "Used, Vintage & Consignment"):
            
            single_label = multi_label
        else:
            single_label = multi_label.split(', ')  
        
    return single_label

In [9]:
def getCategoryCheckinOccurrencesOtm(poiCategory, df, binPois):
    
    categoryOccurrences = 0
    checkinCount = 0
    
    #Para todos os pois no bin, obter as categorias e se for igual ao parâmetro, contar
    for poi, ds in binPois:

        row = df.loc[poi]
        #categories = row['categories'].split(', ') #Obtendo categorias do lugar
        categories = generateSingleLabel(row['categories'])
        for category in categories:
            if(category == poiCategory):
                categoryOccurrences = categoryOccurrences + 1
                checkinCount = checkinCount + row['checkin_count']
                break
    return [categoryOccurrences, checkinCount]

In [12]:
def getTotalCheckinOtm(centerPoI, PoIs, distance, binSize):


    neighbours = PoIs[["latitude", "longitude"]].values
    #print(neighbours)
    center = [[centerPoI[1]['latitude'], centerPoI[1]['longitude']]]
    centers_radians =  np.radians(center)
    neighbours_radians = np.radians(neighbours)
    

    tree = BallTree(neighbours_radians, leaf_size=15, metric='haversine')
    #distance_in_km = distance
    earth_radius_in_km = (6371) # km - earths's radius
    radius = distance / earth_radius_in_km
    is_within, distances = tree.query_radius(centers_radians, r=radius, count_only=False, return_distance=True)

    distance_km = distances * earth_radius_in_km #Transformando as distâncias em m
    distance_m = distance_km * 1000

    checkin_count = 0
    poi_total = 0

    binPois = list()

    #print(distance, is_within, distances)

    for wth, dst in zip(is_within, distance_m):
        #print(wth, dst)
        

        for w in wth:
            
            ds = dst[j]
           
            if((binSize[0] <= ds) and (ds < binSize[1])):
                   
                row = df.loc[w]
                checkin_count = checkin_count + row['checkin_count']
                poi_total = poi_total + 1

                if(w != centerPoI[0]):
                    binPois.append(np.array([w, ds], dtype=object))

        
            j = j + 1
    
    return [checkin_count, poi_total, binPois]

# ITDL

In [16]:
def calculateBin(df, n, h = 100, w=0.5):
    
    print("executing bin:", n, "\th:", h, "\tweight:", w)

    scITDL = {} 
    i = 0 

    binSize = [(n*h),  ((n + 1)*h)] 
    

    for li in df.iterrows():

        tli = generateSingleLabel(li[1]['categories'])

        [sc, sp, binPois] = getTotalCheckinOtm(li, df, (binSize[1]/1000), binSize) #O método espera n em km e ele deve ser passado em m

        sc = round(sc, 0)
        sp = round(sp, 0)

        if (sc != 0 and sp != 0):

            for lj in df.iterrows():

                if(li[0] != lj[0]): #Condição para não inserir o mesmo PoI

                    tlj = generateSingleLabel(lj[1]['categories'])

                    distance = np.inf

                    for poi, ds in binPois:
                        if (poi == lj[0]):
                            distance = ds
                            break


                    if(distance != np.inf):


                        for tki in tli:
                            for tkj in tlj:

                                [cc, cp] = getCategoryCheckinOccurrencesOtm(tkj, df, binPois)

                                if(cp != 0):

                                    a = (1 - (cc/sc)) 
                                    u = (cp/sp)       

                                    if((a > 0) and (u > 0)):

                                        A = -np.log2(a)
                                        U = -np.log2(u)

                                        aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            scITDL[i] = {'business_id_center': li[1]['business_id'],
                                                         #'latitude_centerPoI': li[1]['latitude'],
                                                         #'longitude_centerPoI': li[1]['longitude'],
                                                         'centerPoI': tki,
                                                         'business_id_context': lj[1]['business_id'],
                                                         #'latitude_contextPoI': lj[1]['latitude'],
                                                         #'longitude_contextPoI': lj[1]['longitude'],
                                                         'contextPoI': tkj, 
                                                         'distance-m': distance}
                                            i = i + 1

        
    scITDL = pd.DataFrame.from_dict(scITDL, 'index')
    name = './austin-sl-tuple-n-itdl-' + str(n) + 'bin-wgt'+str(w)+'-p.csv'
    print('saving bin: ', name)
    scITDL.to_csv(name, index = False)
    
    return None

In [13]:
df = pd.read_csv('./austin-ml-sb-updated.csv')
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(2401, 7)
(2401, 7)


,business_id,city,state,latitude,longitude,categories,checkin_count
0,s8eTWEtW5WcnL2kUxrOVmw,Austin,TX,30.401657,-97.742943,"Diagnostic Services, Doctors, Internal Medicin...",139
1,D1d5rpGMpEiq3r1FqgofIg,Austin,TX,30.351343,-97.750405,"Doctors, Health & Medical, Internal Medicine, ...",0
2,n2cW_ZagNpvdc43s0arc2w,Austin,TX,30.418404,-97.849560,"Pediatricians, Family Practice, Doctors, Obste...",37
3,2_l5sU5BBoWeX3J5Yy55Ng,Austin,TX,30.272691,-97.737681,"Internal Medicine, Endocrinologists, Doctors, ...",0
4,0SdT9DBDXGtrgPg94HFRwA,Austin,TX,30.408500,-97.703454,"Medical Spas, Beauty & Spas, Doctors, Health &...",5


In [ ]:
for n in range(0, 27):
    calculateBin(df, n, h = 100, w=0.5)